# Importar bibliotecas

In [1]:
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

# Ler o estoque

In [3]:
patt = './*/*Estoque*.csv'
list_files = glob(patt)

In [4]:
file = list_files[0]

In [5]:
# Colunas que contêm datas.
colunas_data = [
    'DataEmissao',
    'DataAquisicao',
    'DataVencimento',
    'DataGeracao'
]

colunas_texto = [
    'Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf', 'TIT_CEDENTE_ENT_CODIGO',
    'CedenteNome', 'Cnae', 'SecaoCNAEDescricao', 'NotaPdd', 'SAC_TIPO_PESSOA',
    'SacadoCnpjCpf', 'SacadoNome', 'IdTituloVortx', 'TipoAtivo', 'NumeroBoleto',
    'NumeroTitulo', 'CampoChave', 'PagamentoParcial', 'Coobricacao',
    'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3', 'CampoAdicional4',
    'CampoAdicional5', 'IdTituloVortxOriginador', 'Registradora',
    'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio'
]

#*Numéricas: 
colunas_numericas = [
    'ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDNota', 'PDDVencido',
    'PDDTotal', 'PDDEfeitoVagao', 'PercentagemEfeitoVagao'
]

# dic pra ler as de texto
dtype_texto = {col: str for col in colunas_texto}


In [6]:
%%time
dfs = []
for file in list_files:
    df_ = pd.read_csv(file, sep=';', encoding='latin1', dtype=dtype_texto, 
                      decimal=',', parse_dates=colunas_data, dayfirst=True)
    dfs.append(df_)

CPU times: user 7.84 s, sys: 691 ms, total: 8.54 s
Wall time: 8.72 s


In [7]:
%%time
df_final = pd.concat(dfs)

CPU times: user 5.99 s, sys: 50 ms, total: 6.04 s
Wall time: 6.04 s


In [22]:
# criar colunas auxiliares
df_final['_ValorLiquido'] = df_final['ValorPresente'] - df_final['PDDTotal']
df_final['_ValorVencido'] = (df_final['DataVencimento'] <= df_final['DataGeracao']).astype('int') * df_final['ValorPresente']

# sacados com muitos contratos
sacado_contratos = df_final.groupby('SacadoNome')['CCB'].nunique()
k = 3 # numero alto
mask = sacado_contratos[sacado_contratos >= k]
sacado_contratos_alto = sacado_contratos.index[mask]
df_final['_MuitosContratos'] = df_final['SacadoNome'].isin(sacado_contratos_alto).astype('str')

# sacados com muitos entes
sacados_entes = df_final.groupby('SacadoCnpjCpf')['Convênio'].nunique()
k2 = 3
mask = sacados_entes >= k2
sacados_entes_alto = sacados_entes.index[mask]
df_final['_MuitosEntes'] = df_final['SacadoCnpjCpf'].isin(sacados_entes_alto).astype('str')


In [16]:
df_final.memory_usage(deep=True).sum()/1024**2

3049.7270975112915

In [27]:
# Tamanho do estoque
print(f'{df_final["ValorPresente"].sum():_.2f}')

199_232_147.56


# Analisar com `value_counts()`
### Comentários / dúvidas
1. [Situacao] - diferença entre sem cobrança e aditado
2. [SAC_TIPO_PESSOA] - tipo J = jurídico?? tem isso?
3. [SacadoCnpjCpf] - por que tem CNPJ?
4. [SacadoNome] 'BMP SOCIEDADE DE CREDITO DIRETO S.A'
5. [SacadoCnpjCpf'] - verificar consistência dos CPFs
6. [TipoAtivo] - CCB e Contrato. Por que tem contrato?
7. [DataGeracao] - data de referência ou data de processamento?
8. [SacadoCnpjCpf] - tem sacado com 1040 linhas (!)
9. [Convênio] - sacados com muitos convênios (3)

In [19]:
df_final2 = df_final[~df_final['Situacao'].isna()].copy()

for col in df_final2.columns:
    if not df_final2[col].dtype == df_final2['Situacao'].dtype:
        continue
    print(df_final2[col].value_counts(dropna=False))
    print('*'*80)
    

Situacao
Sem cobranÃ§a    1471376
Aditado            50939
Name: count, dtype: int64
********************************************************************************
PES_TIPO_PESSOA
J    1522315
Name: count, dtype: int64
********************************************************************************
CedenteCnpjCpf
34.337.707/0001-00    1519907
92.874.270/0001-40       2408
Name: count, dtype: int64
********************************************************************************
TIT_CEDENTE_ENT_CODIGO
318853     1519907
3224693       2408
Name: count, dtype: int64
********************************************************************************
CedenteNome
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A    1519907
BANCO DIGIMAIS S.A.                                  2408
Name: count, dtype: int64
********************************************************************************
Cnae
6499999    1519907
6422100       2408
Name: count, dtype: int64
********************************************

# Verificando as 117.984 linhas com dados nulos
Pela análise abaixo, não tem influência no cálculo

In [32]:
df_final[df_final['Situacao'].isna()].apply(lambda x: (min(x), max(x)))

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
v1_ = df_final['ValorPresente'].sum() 
v2_ = df_final[~df_final['Situacao'].isna()]['ValorPresente'].sum()
np.isclose(v1_, v2_)

True

In [45]:
del df_final

# Sacado == 'J'
aqui não tem problema

In [62]:
df_sacado_J = df_final2[df_final2['SAC_TIPO_PESSOA'] == 'J'].sample(5)

In [69]:
df_sacado_J['SacadoCnpjCpf'].map(len).max()

14

# Sacado com CNPJ

In [70]:
set(df_final2['SacadoCnpjCpf'].apply(len))

{14, 18}

In [73]:
df_final2_cnpj = df_final2[df_final2['SacadoCnpjCpf'].map(len)==18]
df_final2_cnpj['SacadoNome'].unique()

array(['BMP SOCIEDADE DE CREDITO DIRETO S.A'], dtype=object)

In [77]:
for col in df_final2_cnpj.columns:
    if not df_final2_cnpj[col].dtype == df_final2_cnpj['Situacao'].dtype:
        continue
    print(df_final2_cnpj[col].value_counts(dropna=False))
    print('*'*80)

Situacao
Sem cobranÃ§a    95
Name: count, dtype: int64
********************************************************************************
PES_TIPO_PESSOA
J    95
Name: count, dtype: int64
********************************************************************************
CedenteCnpjCpf
34.337.707/0001-00    95
Name: count, dtype: int64
********************************************************************************
TIT_CEDENTE_ENT_CODIGO
318853    95
Name: count, dtype: int64
********************************************************************************
CedenteNome
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A    95
Name: count, dtype: int64
********************************************************************************
Cnae
6499999    95
Name: count, dtype: int64
********************************************************************************
SecaoCNAEDescricao
OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃO ESPECIFICADAS ANTERIORMENTE    95
Name: count, dtype: int64
******************

In [80]:
df_final2_cnpj.describe(include=[np.number])

,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PDDTotal,PDDEfeitoVagao,PercentagemEfeitoVagao
count,95.000000,9.500000e+01,95.000000,95.0,95.000000,95.000000,95.000000,95.0
mean,23.177368,5.613000e+01,27.604434,0.0,1.650542,27.604434,27.604434,1.0
std,13.054230,1.428624e-14,15.121244,0.0,8.263526,15.121244,15.121244,0.0
min,7.310000,5.613000e+01,8.795000,0.0,0.000000,8.795000,8.795000,1.0
25%,12.005000,5.613000e+01,14.445900,0.0,0.000000,14.445900,14.445900,1.0
50%,19.710000,5.613000e+01,23.720600,0.0,0.000000,23.720600,23.720600,1.0
75%,32.350000,5.613000e+01,38.939150,0.0,0.000000,38.939150,38.939150,1.0
max,53.070000,5.613000e+01,56.130000,0.0,56.130000,56.130000,56.130000,1.0


In [82]:
df_final2_cnpj.select_dtypes(include=[np.number]).sum()

ValorAquisicao            2201.8500
ValorNominal              5332.3500
ValorPresente             2622.4212
PDDNota                      0.0000
PDDVencido                 156.8015
PDDTotal                  2622.4212
PDDEfeitoVagao            2622.4212
PercentagemEfeitoVagao      95.0000
dtype: float64

In [83]:
# Exportar para excel
df_final2_cnpj.to_excel('df_final2_cnpj.xlsx')

# SacadoCnpjCpf - verificar consistência dos CPFs

In [84]:
[implementar]

NameError: name 'implementar' is not defined

df_final2.sample(3)

# Calcular % PDD em função de variáveis categóricas

In [23]:
cat_cols = ['Situacao', 'CedenteNome', 'SAC_TIPO_PESSOA', 'PagamentoParcial', 'TipoAtivo', '_MuitosContratos', '_MuitosEntes']

for col in cat_cols:
    aux_ = df_final2.groupby(col)[['_ValorLiquido', 'ValorPresente']].sum()
    aux_['%PDD'] = 1- aux_['_ValorLiquido'] / aux_['ValorPresente']
    display(aux_)
    

,_ValorLiquido,ValorPresente,%PDD
Situacao,,,
Aditado,7.851224e+06,8.081707e+06,0.028519
Sem cobranÃ§a,1.516009e+08,1.911504e+08,0.206903


,_ValorLiquido,ValorPresente,%PDD
CedenteNome,,,
BANCO DIGIMAIS S.A.,2.469849e+05,2.972629e+05,0.169137
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,1.592051e+08,1.989349e+08,0.199713


,_ValorLiquido,ValorPresente,%PDD
SAC_TIPO_PESSOA,,,
F,1.300191e+08,1.506257e+08,0.136807
J,2.943296e+07,4.860642e+07,0.394464


,_ValorLiquido,ValorPresente,%PDD
PagamentoParcial,,,
NAO,1.593906e+08,1.989932e+08,0.199015
SIM,6.149045e+04,2.389057e+05,0.742616


,_ValorLiquido,ValorPresente,%PDD
TipoAtivo,,,
CCB,1.509618e+08,1.902773e+08,0.206622
CT - Contrato,8.490281e+06,8.954897e+06,0.051884


,_ValorLiquido,ValorPresente,%PDD
_MuitosContratos,,,
False,1.591793e+08,1.989402e+08,0.199864
True,2.728016e+05,2.919855e+05,0.065701


,_ValorLiquido,ValorPresente,%PDD
_MuitosEntes,,,
False,1.589462e+08,1.985388e+08,0.199420
True,5.058947e+05,6.933760e+05,0.270389


# Repetir o item anterior para vencidos

In [24]:
for col in cat_cols:
    aux_ = df_final2.groupby(col)[['_ValorVencido', 'ValorPresente']].sum()
    aux_['%Vencido'] = (aux_['_ValorVencido'] / aux_['ValorPresente']) * 100
    display(aux_)
    

,_ValorVencido,ValorPresente,%Vencido
Situacao,,,
Aditado,38818.61,8.081707e+06,0.480327
Sem cobranÃ§a,10165952.80,1.911504e+08,5.318299


,_ValorVencido,ValorPresente,%Vencido
CedenteNome,,,
BANCO DIGIMAIS S.A.,19910.86,2.972629e+05,6.698064
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,10184860.55,1.989349e+08,5.119696


,_ValorVencido,ValorPresente,%Vencido
SAC_TIPO_PESSOA,,,
F,6282185.49,1.506257e+08,4.170725
J,3922585.92,4.860642e+07,8.070099


,_ValorVencido,ValorPresente,%Vencido
PagamentoParcial,,,
NAO,9967888.88,1.989932e+08,5.009160
SIM,236882.53,2.389057e+05,99.153147


,_ValorVencido,ValorPresente,%Vencido
TipoAtivo,,,
CCB,9932005.12,1.902773e+08,5.219754
CT - Contrato,272766.29,8.954897e+06,3.046001


,_ValorVencido,ValorPresente,%Vencido
_MuitosContratos,,,
False,10197323.25,1.989402e+08,5.125824
True,7448.16,2.919855e+05,2.550866


,_ValorVencido,ValorPresente,%Vencido
_MuitosEntes,,,
False,10178132.62,1.985388e+08,5.126521
True,26638.79,6.933760e+05,3.841897


# Sacados em mais de um ente?

In [29]:
# SacadoCnpjCpf , SacadoNome
sacados_entes = df_final2.groupby('SacadoCnpjCpf')['Convênio'].agg([pd.Series.nunique, pd.unique])
sacados_entes.sort_values('nunique', ascending=False)[:35]

,nunique,unique
SacadoCnpjCpf,,
048.692.204-90,3,"[GOV. RIO GRANDE DO NORTE, GOV. GOIAS, PREF. T..."
323.026.574-20,3,"[GOV. RIO GRANDE DO NORTE, GOV. GOIAS, GOV. SÃ..."
292.798.363-15,2,"[GOV. MARANHAO, PREF. BALSAS]"
443.808.794-53,2,"[GOV. RIO GRANDE DO NORTE, PREF. SÃO JOSÉ DE M..."
813.733.763-68,2,"[GOV. MARANHAO, PREF. BALSAS]"
647.592.703-10,2,"[GOV. MARANHAO, PREF. SÃO LUIS]"
082.249.978-98,2,"[GOV. RIO GRANDE DO NORTE, PREF. SÃO JOSÉ DE M..."
054.906.964-00,2,"[GOV. RIO GRANDE DO NORTE, PREF. CAMPOS DOS GO..."
268.919.048-62,2,"[PREF. MAUÁ, PREF. SANTO ANDRE]"


In [101]:
df_final2.columns

Index(['Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf',
       'TIT_CEDENTE_ENT_CODIGO', 'CedenteNome', 'Cnae', 'SecaoCNAEDescricao',
       'NotaPdd', 'SAC_TIPO_PESSOA', 'SacadoCnpjCpf', 'SacadoNome',
       'IdTituloVortx', 'TipoAtivo', 'DataEmissao', 'DataAquisicao',
       'DataVencimento', 'NumeroBoleto', 'NumeroTitulo', 'CampoChave',
       'ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDNota',
       'PDDVencido', 'PagamentoParcial', 'Coobricacao', 'DataGeracao',
       'PDDTotal', 'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3',
       'CampoAdicional4', 'CampoAdicional5', 'PDDEfeitoVagao',
       'PercentagemEfeitoVagao', 'IdTituloVortxOriginador', 'Registradora',
       'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio',
       '_ValorLiquido'],
      dtype='object')

In [148]:
df_final2.sample(2)

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos
608870,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,565.131.602-78,ISABEL GIANE COELHO RODRIGUES,100015359,CCB,2025-06-13,2025-06-16,2030-07-15,NaN,57139775060,57139775060,125.10,931.37,128.9142,0.0,0.0,NAO,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,57139775,PREF. JURUTI,128.9142,0.0,False
343873,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,490.381.254-53,JOACILDO DE MEDEIROS GALVAO,41173845,CCB,2024-05-29,2024-05-31,2028-11-16,NaN,3620565153,155164184,8.88,30.39,12.0912,0.0,0.0,NAO,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,36205651,GOV. RIO GRANDE DO NORTE,12.0912,0.0,False
